### get accuracy of sivqa evaluation

idefics2 model

In [1]:
import utils, sivqa_utils
import os 
import json
from sklearn.metrics import accuracy_score
import re
import random

In [2]:
result_dir = "/scratch3/wenyan/data/foodie/results"

In [3]:
data = []
with open(os.path.join(result_dir, "sivqa_idefics_prompt3.jsonl"), "r", encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

In [4]:
sivqa = sivqa_utils.read_sivqa("/scratch3/wenyan/data/foodie")

In [50]:

ans2idx = {
        "A":"0",
        "B":"1",
        "C":"2",
        "D":"3"
        }

def parse_res(res):
    ans_str = res["response"][0].split("\nAssistant:")[-1].strip()
    ans_letter = re.findall(r'[A-Z]', ans_str)
    if not ans_letter or len(ans_letter) == 0:
        print("can not parse ans for res: ", res)
        return random.choice(["0", "1", "2", "3"])
    else:
        ans = ans_letter[0].upper()
        if ans not in ans2idx:
            print("can not parse ans for res: ", res)
            return random.choice(["0", "1", "2", "3"])
        else:
            return ans2idx[ans]


In [51]:
gts = [s["answer"] for s in sivqa]
answers = [parse_res(d) for d in data]
accuracy = accuracy_score(gts, answers)
print(accuracy)

can not parse ans for res:  {'response': ['User: 图片中的食物通常属于哪个菜系? 这是选项: （A) 赣菜 (客家菜）\n（B) 内蒙古菜\n（C) 粤菜\n（D) 赣菜\n 请根据上图从所提供的选项中选择一个正确答案。 \nAssistant: 我选择（ \nAssistant: 客家菜).'], 'qid': 'vqa-193'}
0.46875


get qwen accuracy

In [28]:
data = []
with open(os.path.join(result_dir, "sivqa_qwen_prompt2.jsonl"), "r", encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

In [29]:
data[0]["response"]#.split("为")[-1].strip()

'Picture 1:<img>/scratch3/wenyan/data/foodie/14456664_all_202404292352223293/217_IMG_3854.jpeg</img>\n用户：你是一个智能助手，现在请看图回答以下选择题：图片中的食物通常属于哪个菜系? 选项有: （A) 京菜\n（B) 徽菜\n（C) 新疆菜\n（D) 桂菜\n智能助手：我从所提供的选项中选择一个正确答案，为（C) 新疆菜。<|endoftext|>'

In [30]:

ans2idx = {
        "A":"0",
        "B":"1",
        "C":"2",
        "D":"3"
        }

def parse_res(res, template=0):
    if template == 0 or template ==1 or template == 3:
        ans_str = res["response"].split("选择一个正确答案，")[1].strip()
    ans_letter = re.findall(r'([A-Z])', ans_str)
    if not ans_letter or len(ans_letter) == 0:
        print("can not parse ans for res: ", res)
        return random.choice(["0", "1", "2", "3"])
    else:
        ans = ans_letter[0].upper()
        if ans not in ans2idx:
            print("can not parse ans for res: ", res)
            return random.choice(["0", "1", "2", "3"])
        else:
            return ans2idx[ans]


In [31]:
parse_res(data[0])

'2'

In [32]:
def get_accuracy(data, sivqa, template=0):
    gts = [s["answer"] for s in sivqa]
    answers = [parse_res(d, template=template) for d in data]
    accuracy = accuracy_score(gts, answers)
    print(accuracy)

In [33]:
get_accuracy(data, sivqa, template=0)

0.48828125
